<a href="https://colab.research.google.com/github/Weimw/movielens-project/blob/main/CS224w_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install Packages and Libraries

In [2]:
# Install required packages.
%%capture
import torch
import os
os.environ['TORCH'] = torch.__version__
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [23]:
# import required modules
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import structured_negative_sampling

## 2. Data Preprocessing

In [4]:
# download the dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

movie_path = './ml-latest-small/movies.csv'
rating_path = './ml-latest-small/ratings.csv'

Extracting ./ml-latest-small.zip


The `ratings.csv` data connects users (as given by `userId`) and movies (as given by `movieId`). We first want to create a mapping that maps entry IDs to a consecutive value in the range `{ 0, ..., num_rows - 1 }`. This is very helpful because it will make the adjency matrix as compact as possible. We then create an `edge_index` which is of shape `[2, num_ratings]` from `ratings.csv`. The first row represents the mapped `userId` and the second row represents the mapped `movieId`. A specific column represents a connection (i.e rating) between a user and a movie. The `edge_index` tensor is very important when propagating the messages along the graph network. 

In [5]:
def preprocessing(movie_path: str, rating_path: str): 
  '''
    Args:
         movie_path (str): A string representing the file path to the movies dataset.
         rating_path (str): A string representing the file path to the ratings dataset.
    
    Returns:
         edge_index (torch.Tensor): the indices of edges in the adjacency matrix for the ratings dataset.
         num_users (int): number of unique users in the ratings dataset.
         num_movies (int): number of unique movies in the ratings dataset.
  '''
  # load movies and ratings dataset
  movie_df = pd.read_csv(movie_path, index_col = 'movieId')
  rating_df = pd.read_csv(rating_path, index_col = 'userId') 

  # create mapping to continous range
  movie_mapping = {idx: i for i, idx in enumerate(movie_df.index.unique())}
  user_mapping = {idx: i for i, idx in enumerate(rating_df.index.unique())}
  num_users, num_movies = len(rating_df.index.unique()), len(movie_df.index.unique())

  rating_df = pd.read_csv(rating_path)
  edge_index = None
  users = [user_mapping[idx] for idx in rating_df['userId']]
  movies = [movie_mapping[idx] for idx in rating_df['movieId']]

  # filter for edges with a high rating
  ratings = rating_df['rating'].values
  recommend_bool = torch.from_numpy(ratings).view(-1, 1).to(torch.long) >= 4

  edge_index = [[],[]]
  for i in range(recommend_bool.shape[0]):
    if recommend_bool[i]:
      edge_index[0].append(users[i])
      edge_index[1].append(movies[i])
    
  edge_index = torch.tensor(edge_index)
  return edge_index, num_users, num_movies

In [6]:
edge_index, num_users, num_movies = preprocessing(movie_path, rating_path)

We then split the whole dataset into train/val/test sets. The split is conducted on edge level. To be more specifc, we will split the whole dataset into 80/10/10 by random, so that each set will have a subset of `edge_index`.

In [24]:
num_ratings = edge_index.shape[1]
rating_indices = np.arange(num_ratings)

indices_train, indices_val_test = train_test_split(rating_indices, test_size = 0.2, random_state = 42)
indices_val, indices_test = train_test_split(indices_val_test, test_size = 0.5, random_state = 42)

# slice the whole dataset by split indices, then convert to SparseTensor for later training
def generate_edge(edge_indices: np.ndarray):
  '''
  Args:
      edge_indices (np.ndarray): An array representing the indices of edges in the dataset.

  Returns:
      sub_edge_index (torch.Tensor): indices of edges in the specified subset.
      edge_index_sparse (SparseTensor): A sparse tensor representing the adjacency matrix for the subset of edges.
  '''
  sub_edge_index = edge_index[:, edge_indices]
  num_nodes = num_users + num_movies
  edge_index_sparse = SparseTensor(row = sub_edge_index[0],
                                   col = sub_edge_index[1],
                                   sparse_sizes = (num_nodes, num_nodes))
  return sub_edge_index, edge_index_sparse

train_edge_index, train_sparse_edge_index = generate_edge(indices_train)
val_edge_index, val_sparse_edge_index = generate_edge(indices_val)
test_edge_index, test_sparse_edge_index = generate_edge(indices_test)

In training process, we will use mini-batch strategy and sample several positive edges and negative edges within each batch. A positive edge is defined as observed/training user-item interactions. During training, we want to penalize those negative edges by assigning larger loss to them. We will utilize the `structured_negative_sampling` function in PyG, which samples a negative edge for every positive edge in the graph, given by `edge_index`.

In [8]:
'''
samples a negative edge :obj:`(i,k)` for every positive edge :obj:`(i,j)` in the 
graph given by :attr:`edge_index`, and returns it as a tuple of the form :obj:`(i,j,k)`.
'''
edges = structured_negative_sampling(train_edge_index)
edges = torch.stack(edges, dim=0)
edges

tensor([[ 408,  579,    2,  ...,  482,   15,  205],
        [ 863,  990, 3734,  ..., 5729,  561,  621],
        [5240,  130, 5840,  ..., 9687, 6869, 7832]])

In [9]:
# function which random samples a mini-batch of positive and negative samples
def sample_mini_batch(batch_size, edge_index):
    """Randomly samples indices of a minibatch given an adjacency matrix

    Args:
        batch_size (int): minibatch size
        edge_index (torch.Tensor): 2 by N list of edges

    Returns:
        tuple: user indices, positive item indices, negative item indices
    """
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    indices = torch.randperm(edges.shape[1])[:batch_size]
    batch = edges[:, indices]
    user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
    return user_indices, pos_item_indices, neg_item_indices

## 3.Model Architecture

## Light Graph Convolution
Between each layer, LightGCN uses the following propagation rule for user and item embeddings.

\begin{equation}
e_u^{(k+1)} = \sum_{i \in N_u} \frac{1}{\sqrt{|N_u|}\sqrt{|N_i|}} e_i^{(k)} \quad e_i^{(k+1)} = \sum_{u \in N_i} \frac{1}{\sqrt{|N_i|}\sqrt{|N_u|}} e_u^{(k)}
\end{equation}

$N_u$: the set of all neighbors of user $u$ (items liked by $u$)

$N_i$: the set of all neighbors of item $i$ (users who liked $i$)

$e_u^{(k)}$ : k-th layer user embedding

$e_i^{(k)}$ : k-th layer item embedding



## Layer Combination and Model Prediction
The only trainable parameters of LightGCN are the 0-th layer embeddings $e_u^{(0)}$ and $e_i^{(0)}$ for each user and item. We combine the embeddings obtained at each layer of propagation to form the final embeddings for all user and item, $e_u$ and $e_i$ via the follwing equation.


\begin{equation}
e_u = \sum_{k = 0}^K \alpha_k e_u^{(k)} \quad e_i = \sum_{k = 0}^K \alpha_k e_i^{(k)}
\end{equation}

$\alpha_k$ : hyperparameter which weights the contribution of the k-th layer embedding to the final embedding

The model prediction is obtained by taking the inner product of the final user and item embeddings.

\begin{equation}
\hat{y}_{ui} = e_u^Te_i
\end{equation}

## Matrix Form
In our implementation, we utilize the matrix form of LightGCN. We perform multi-scale diffusion to obtain the final embedding, which sums embeddings diffused across multi-hop scales. 

\begin{equation}
E^{(K)} = \alpha_0 E^{(0)} + \alpha_1 \tilde{A}^1 E^{(0)} + \alpha_2 \tilde{A}^2 E^{(0)} + \cdot \cdot \cdot + \alpha_K \tilde{A}^K \tilde{A} E^{(0)}
\end{equation}

$E^{(0)} \in \mathcal{R}^{(M + N)} \times T$ : stacked initial item and user embeddings where $M$, $N$, and $T$ denote the number of users, number of items, and the dimension of each embedding respectively

$\tilde{A} = D^{-\frac{1}{2}}AD^{-\frac{1}{2}}$ : symmetrically normalized adjacency matrix



In [10]:
# defines LightGCN model
class LightGCN(MessagePassing):
    """
    LightGCN Model, see reference: https://arxiv.org/abs/2002.02126
    We omit a dedicated class for LightGCNConvs for easy access to embeddings
    """

    def __init__(self, num_users: int, num_items: int, hidden_dim: int, num_layers: int):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.users_emb = nn.Embedding(self.num_users, self.hidden_dim) # e_u^0
        self.items_emb = nn.Embedding(self.num_items, self.hidden_dim) # e_i^0

        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)

    def forward(self, edge_index: SparseTensor):
        """
        Forward pass of the LightGCN model. Returns the init and final
        embeddings of the user and item
        """
        edge_index_norm = gcn_norm(edge_index, False)

        # The first layer, concat embeddings
        x0 = torch.cat([self.users_emb.weight, self.items_emb.weight])
        xs = [x0]
        xi = x0

        # pass x to the next layer
        for i in range(self.num_layers):
            xi = self.propagate(edge_index_norm, x=xi)
            xs.append(xi)

        xs = torch.stack(xs, dim=1)
        x_final = torch.mean(xs, dim=1)

        users_emb, items_emb = \
        torch.split(x_final, [self.num_users, self.num_items])

        return users_emb, self.users_emb.weight, items_emb, self.items_emb.weight

    def message(self, x: Tensor) -> Tensor:
        return x

    def propagate(self, edge_index: SparseTensor, x: Tensor) -> Tensor:
        x = self.message_and_aggregate(edge_index, x)
        return x

    def message_and_aggregate(self, edge_index: SparseTensor, x: Tensor) -> Tensor:
        return matmul(edge_index, x)


## Loss Function



We utilize a Bayesian Personalized Ranking (BPR) loss, a pairwise objective which encourages the predictions of positive samples to be higher than negative samples for each user.

\begin{equation}
L_{BPR} = -\sum_{u = 1}^M \sum_{i \in N_u} \sum_{j \notin N_u} \ln{\sigma(\hat{y}_{ui} - \hat{y}_{uj})} + \lambda ||E^{(0)}||^2 
\end{equation}

$\hat{y}_{u}$: predicted score of a positive sample

$\hat{y}_{uj}$: predicted score of a negative sample

$\lambda$: hyperparameter which controls the L2 regularization strength

In [26]:
def bpr_loss(users_emb, user_emb_0, pos_emb, pos_emb_0, neg_emb, neg_emb_0, lambda_val):
  # calculate the Bayesian Personalized Ranking from the embeddings 
    pos_scores = torch.sum(users_emb * pos_emb, dim=1) 
    neg_scores = torch.sum(users_emb * neg_emb, dim=1)
    losses = -torch.log(torch.sigmoid(pos_scores - neg_scores))
    loss = torch.mean(losses) + lambda_val * \
    (torch.norm(users_emb_0) + torch.norm(pos_emb_0) + torch.norm(neg_emb_0))
    
    return loss

## 4.Evaluation Metrics

The evaluation metric we will use is **Recall@K**, it is defined as the proportion of relevant items that are recommended to a user among the top-K items recommended by the algorithm. \\
For each user $u$, \\
Let $P_{u}$ be a set of positive items the user will interact in the future. \\
Let $R_{u}$ be a set of items recommended by the model, in top-K recommendation, $|R_{u}| = K$. Items that users has already interacted are excluded.

![picture](https://drive.google.com/uc?id=1Ea3_y0eLNNKZT2p13Sa-umZccVuNtRvo)

**Recall@K** for user $u$ is $|P_{u}\cap R_{u}| / |P_{u}|$. \\
The final Recall@K is computed by averaging the recall values across all users.


In [27]:
def get_user_positive_items(edge_index):
    """
    Return positive items for all users in form of list
    """
    user_pos_items = {}
    for i in range(edge_index.shape[1]):
        user = edge_index[0][i].item()
        item = edge_index[1][i].item()
        if user not in user_pos_items:
            user_pos_items[user] = []
        user_pos_items[user].append(item)
    return user_pos_items

In [28]:
def recallAtK(actual_r, pred_r, k):
    """
    Return recall at k and precision at k
    """
    correct_count = torch.sum(pred_r, dim=-1)
    # number of items liked by each user in the test set
    liked_count = torch.Tensor([len(actual_r[i]) for i in range(len(actual_r))])
    
    recall = torch.mean(correct_count / liked_count)
    precision = torch.mean(correct_count) / k
    
    return recall.item(), precision.item()


In [38]:
def get_metrics(model, edge_index, masked_indices, k):

    users_emb = model.users_emb.weight
    items_emb = model.items_emb.weight

    # set ratings matrix between every user and item 
    rating = torch.matmul(users_emb, items_emb.T)

    for index in masked_indices:
        user_pos_items = get_user_positive_items(index)
        masked_users = []
        masked_items = []
        for user, items in user_pos_items.items():
            masked_users.extend([user] * len(items))
            masked_items.extend(items)

        rating[masked_users, masked_items] = float("-inf")

    # top k recommended items for each user
    _, top_K_items = torch.topk(rating, k=k)

    # get all unique users and ratings for eval
    users = edge_index[0].unique()
    test_user_pos_items = get_user_positive_items(edge_index)

    actual_r = [test_user_pos_items[user.item()] for user in users]
    pred_r = []

    for user in users:
        items = test_user_pos_items[user.item()]
        label = list(map(lambda x: x in items, top_K_items[user]))
        pred_r.append(label)
    
    pred_r = torch.Tensor(np.array(pred_r).astype('float'))
    
    return recallAtK(actual_r, pred_r, k)
   

In [39]:
def evaluation(model, edge_index, sparse_edge_index, mask, k, lambda_val):
    """
    Evaluates model loss and metrics including recall, precision
    @args: 
        model: lightgcn model
        edge_index: edges for split to evaluate
        sparse_edge_index: sparse adjacency matrix 
        mask: edges for split to remove from eval, in form of list
        k: top k item
    @return
    """
    # get embeddings
    users_emb, users_emb_0, items_emb, items_emb_0 = model.forward(sparse_edge_index)
    edges = structured_negative_sampling(edge_index, contains_neg_self_loops=False)
    
    user_indices, pos_indices, neg_indices = edges[0], edges[1], edges[2]
    users_emb, users_emb_0 = users_emb[user_indices], users_emb_0[user_indices]
    pos_emb, pos_emb_0 = items_emb[pos_indices], items_emb_0[pos_indices]
    neg_emb, neg_emb_0 = items_emb[neg_indices], items_emb_0[neg_indices]

    loss = bpr_loss(users_emb, users_emb_0, pos_emb, pos_emb_0,
                    neg_emb, neg_emb_0, lambda_val).item()

    recall, precision = get_metrics(model, edge_index, mask, k)

    return loss, recall, precision

## 5.Training Process

Your test set performance should be in line with the following (*K=20*):

*Recall@K: 0.13, Precision@K: 0.045, NDCG@K: 0.10*

In [54]:
# model configurations
config = {
    'batch_size': 32,
    'num_epoch': 50,
    'epoch_size': 200,
    'lr': 1e-3,
    'lr_decay': 0.9,
    'topK': 10,
    'lambda': 1e-6,
    'hidden_dim': 16,
    'num_layer': 3,
}

In [52]:
# setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LightGCN(num_users, num_movies, config['hidden_dim'], config['num_layer'])
model = model.to(device)
model.train()

optimizer = optim.Adam(model.parameters(), lr=config['lr'])
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=config['lr_decay'])

edge_index = edge_index.to(device)
train_edge_index = train_edge_index.to(device)
train_sparse_edge_index = train_sparse_edge_index.to(device)

val_edge_index = val_edge_index.to(device)
val_sparse_edge_index = val_sparse_edge_index.to(device)

In [1]:
# training loop
train_losses = []
val_losses = []

for epoch in range(config['num_epoch']):
  for iter in range(config['epoch_size']):
    # forward propagation
    users_emb, users_emb_0, items_emb, items_emb_0 = \
        model.forward(train_sparse_edge_index)

    # mini batching
    user_indices, pos_indices, neg_indices = \
        sample_mini_batch(config['batch_size'], train_edge_index)
    
    user_indices = user_indices.to(device)
    pos_indices = pos_indices.to(device)
    neg_indices = neg_indices.to(device)
    
    users_emb, users_emb_0 = users_emb[user_indices], users_emb_0[user_indices]
    pos_emb, pos_emb_0 = items_emb[pos_indices], items_emb_0[pos_indices]
    neg_emb, neg_emb_0 = items_emb[neg_indices], items_emb_0[neg_indices]

    # loss computation
    loss = bpr_loss(users_emb, users_emb_0, 
                    pos_emb, pos_emb_0,
                    neg_emb, neg_emb_0,
                    config['lambda'])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  model.eval()
  val_loss, recall, precision = evaluation(model, val_edge_index, 
                                           val_sparse_edge_index, 
                                           [train_edge_index], 
                                           config['topK'],
                                           config['lambda'])
  

  print('Epoch {:d}: train_loss: {:.4f}, val_loss: {:.4f}, recall: {:.4f}, precision: {:.4f}'\
        .format(epoch, loss, val_loss, recall, precision))
  train_losses.append(loss.item())
  val_losses.append(val_loss)

  scheduler.step()

NameError: ignored

In [ ]:
iters = [config['epoch_size'] * epoch for epoch in range(config['num_epoch'])]
plt.plot(iters, train_losses, label='Train')
plt.plot(iters, val_losses, label='Validation')
plt.xlabel('Iteration')
plt.ylabel('Losses')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()

In [ ]:
# evaluate on test set
model.eval()
test_edge_index = test_edge_index.to(device)
test_sparse_edge_index = test_sparse_edge_index.to(device)

test_loss, test_recall, test_precision \
    = evaluation(model, 
                test_edge_index, 
                test_sparse_edge_index, 
                [train_edge_index, val_edge_index],
                config['topK'],
                config['lambda'])
    

print('Test set: train_loss: {:.4f}, val_loss: {:.4f}, recall: {:.4f}, precision: {:.4f}'\
        .format(test_loss, test_recall, test_precision))


# Make New Recommendatios for a Given User

In [ ]:
model.eval()
df = pd.read_csv(movie_path)
movieid_title = pd.Series(df.title.values,index=df.movieId).to_dict()
movieid_genres = pd.Series(df.genres.values,index=df.movieId).to_dict()

user_pos_items = get_user_positive_items(edge_index)

In [ ]:
def make_predictions(user_id, num_recs):
    user = user_mapping[user_id]
    e_u = model.users_emb.weight[user]
    scores = model.items_emb.weight @ e_u

    values, indices = torch.topk(scores, k=len(user_pos_items[user]) + num_recs)

    movies = [index.cpu().item() for index in indices if index in user_pos_items[user]][:num_recs]
    movie_ids = [list(movie_mapping.keys())[list(movie_mapping.values()).index(movie)] for movie in movies]
    titles = [movieid_title[id] for id in movie_ids]
    genres = [movieid_genres[id] for id in movie_ids]

    print(f"Here are some movies that user {user_id} rated highly")
    for i in range(num_recs):
        print(f"title: {titles[i]}, genres: {genres[i]} ")

    print()

    movies = [index.cpu().item() for index in indices if index not in user_pos_items[user]][:num_recs]
    movie_ids = [list(movie_mapping.keys())[list(movie_mapping.values()).index(movie)] for movie in movies]
    titles = [movieid_title[id] for id in movie_ids]
    genres = [movieid_genres[id] for id in movie_ids]

    print(f"Here are some suggested movies for user {user_id}")
    for i in range(num_recs):
        print(f"title: {titles[i]}, genres: {genres[i]} ")

In [ ]:
USER_ID = 1
NUM_RECS = 10

make_predictions(USER_ID, NUM_RECS)